 # ARIMA based on top x crimes

In [ ]:
from datetime import datetime

In [ ]:
TOP_X_CRIMES = 1
SAVE_TO_CSV = False
ROUND_PREDICTIONS = False
SAVE_MODEL_TO_DISK = False
SAVE_DICT_TO_DISK = False
CURRENT_DATE = datetime.now().strftime("%d%m%Y_%H%M")
print("CURRENT_DATE", CURRENT_DATE)

## Imports

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import os
import pickle
import tqdm
from PIL import Image
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
import scipy.stats as stats
from scipy.stats import shapiro
from statsmodels.stats.diagnostic import acorr_ljungbox
from pandas.plotting import autocorrelation_plot
from sklearn.metrics import root_mean_squared_error
from math import sqrt
from pmdarima import auto_arima
import seaborn as sns
from collections import defaultdict

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Data

In [ ]:
path = '..\\data\\mergedData\\ts_df.csv'
ts_df = pd.read_csv(path)
ts_df = ts_df.loc[:, ~ts_df.columns.str.contains('^Unnamed')]
ts_df.head()

## Time Series per Crime per District

In [ ]:
dist_ids = ts_df.dist_id.unique()
dist_ids = sorted(dist_ids)
print("dist_ids", dist_ids)
num_dists = len(dist_ids)
print("num_dists", num_dists)

In [ ]:
cluster_names = ts_df.Cluster_Name.unique()
cluster_names = sorted(cluster_names)
print("cluster_names", cluster_names)
num_cluster_names = len(cluster_names)
print("num_cluster_names", num_cluster_names)

In [ ]:
crime_dist_dfs = {}
for i, dist_id in enumerate(dist_ids):
    # Filter data for this district
    dist_data = ts_df[ts_df['dist_id'] == dist_id]
    crime_dist_dfs[dist_id] = {}
    for i, cluster_name in enumerate(cluster_names):
        crime_data = dist_data[dist_data['Cluster_Name'] == cluster_name]
        # Prepare ts data and split
        train = crime_data.loc[crime_data.Reported_Date < "2024-01-01"]
        test = crime_data.loc[crime_data.Reported_Date >= "2024-01-01"]
        train = train.groupby("Reported_Date").size().reset_index(name='Crime_Count')
        test = test.groupby("Reported_Date").size().reset_index(name='Crime_Count')
        train.index = pd.to_datetime(train.Reported_Date)
        test.index = pd.to_datetime(test.Reported_Date)
        train = train.asfreq('d', fill_value=0)
        test = test.asfreq('d', fill_value=0)
        train = train.drop(columns=['Reported_Date'])
        test = test.drop(columns=['Reported_Date'])
        # Save final dict
        crime_dist_dfs[dist_id][cluster_name] = {
            "train": train,
            "test": test
        }

In [ ]:
print(f"number of datasets in dict: {num_dists*num_cluster_names}")
print(f"districts in dict: {crime_dist_dfs.keys()}")
print(f"crimes in dict: {crime_dist_dfs[1.0].keys()}")
print(f"test/train in dict: {crime_dist_dfs[1.0]['Alcohol Influence'].keys()}")

In [ ]:
crime_dist_dfs[1.0]['Alcohol Influence']["train"].head()

### Filter out crimes with low counts

> keep only top x crimes

In [ ]:
def get_top_x_crimes(crime_dist_dfs, X = 10):
    # Step 1: Aggregate crime counts across dates and districts
    total_crime_counts = defaultdict(int)

    for district, crimes in crime_dist_dfs.items():
        for crime, data_dict in crimes.items():
            # Sum the Crime_Count column for the 'train' DataFrame of each crime
            total_crime_counts[crime] += data_dict["train"]["Crime_Count"].sum()

    # Step 2: Sort crimes by count in descending order and extract the top X crimes
    X = X  # Specify the number of top crimes to retrieve
    top_crimes_set = sorted(total_crime_counts.items(), key=lambda x: x[1], reverse=True)[:X]
    
    top_crimes = []
    # Display the top X crimes
    for crime, count in top_crimes_set:
        print(f"{crime}: {count}")
        top_crimes.append(crime)

    # Step 3: Filter `crime_dist_dfs` to retain only the top X crimes
    filtered_crime_dist_dfs = {
        district: {crime: data_dict for crime, data_dict in crimes.items() if crime in top_crimes}
        for district, crimes in crime_dist_dfs.items()
    }

    # `filtered_crime_dist_dfs` now contains only the top X crimes
    return filtered_crime_dist_dfs

In [ ]:
crime_dist_dfs = get_top_x_crimes(crime_dist_dfs, X = TOP_X_CRIMES)

> Save dataframes

In [ ]:
if SAVE_TO_CSV:
    # Base path for saving the data
    base_path = '..\\data\\top_10_crimes\\'

    # Iterate over district IDs, crime types, and data type (train/test)
    for district_id, crimes in crime_dist_dfs.items():
        for crime_name, data in crimes.items():
            
            # Create the folder path for each district and crime type
            crime_path = os.path.join(base_path, str(district_id), crime_name)
            os.makedirs(crime_path, exist_ok=True)  # Create the directories if they don't exist
            
            # Save the train and test dataframes as CSVs
            for data_type, df in data.items():
                file_path = os.path.join(crime_path, f"{data_type}.csv")
                df.to_csv(file_path)
        print(f"saved all files for district {district_id} to disk..")

### Auto arima

In [ ]:
def test_adf_stationary(data):
    result = adfuller(data)

    if result[1] > 0.05:
        return False
    else:
        return True

In [ ]:
if SAVE_MODEL_TO_DISK:
    os.makedirs("Arima_models", exist_ok=True)
    print("Created Arima_models directory..")

# Loop through each district and crime category
for dist_id, crimes in crime_dist_dfs.items():
    for cluster_name, data_splits in tqdm.tqdm(crimes.items(), desc=f"dist {dist_id} Crimes", leave=False):
        train_data = data_splits["train"]
        test_data = data_splits["test"]
        
        try:
            # Check if stationary
            is_stationary = test_adf_stationary(train_data["Crime_Count"])
            
            # Fit the ARIMA model
            #arima_model = auto_arima(train_data, stationary=is_stationary, D=1, start_p=10, max_p=15, start_q=0, max_q=2, seasonal=False, stepwise=True, trace=False, error_action="warn", n_fits=20)
            arima_model = auto_arima(train_data, seasonal=False, stationary=is_stationary, stepwise=True, trace=False, error_action="ignore")

            # Save model to disk
            if SAVE_MODEL_TO_DISK:
                model_store_path = os.path.join("Arima_models", str(dist_id), cluster_name)
                os.makedirs(model_store_path, exist_ok=True)
                print(f"Created: {model_store_path} directory..")
                # Serialize with Pickle
                with open(os.path.join(model_store_path, 'arima.pkl'), 'wb') as pkl:
                    pickle.dump(arima_model, pkl)

            # Save model configuration
            crime_dist_dfs[dist_id][cluster_name]["model_config"] = {
                "p": arima_model.order[0],
                "d": arima_model.order[1],
                "q": arima_model.order[2],
                "aic": arima_model.aic()
            }

            residuals = arima_model.resid()

            # Perform Shapiro-Wilk test on residuals
            shapiro_test_statistic, p_value = shapiro(residuals)

            # Interpretation
            if p_value > 0.05:
                raise Exception("**Residuals are normally distributed (fail to reject H0).**")

            # Determine the number of predictions
            n_predictions = len(test_data) if len(test_data) > 0 else 265
            
            # Generate predictions
            predictions = arima_model.predict(n_periods=n_predictions)

            if ROUND_PREDICTIONS:
                # Round predictions to the nearest whole numbers
                predictions = predictions.round()
            
            # Calculate RMSE if test data is available
            if len(test_data) > 0:
                rmse = root_mean_squared_error(test_data, predictions)
                crime_dist_dfs[dist_id][cluster_name]["model_config"]["rmse"] = rmse
            else:
                crime_dist_dfs[dist_id][cluster_name]["model_config"]["rmse"] = None

            # Save predictions and residuals in data structure
            crime_dist_dfs[dist_id][cluster_name]["forecast"] = predictions
            crime_dist_dfs[dist_id][cluster_name]["residuals"] = residuals
            
        except Exception as e:
            print(f"ARIMA model failed for district {dist_id}, crime {cluster_name} with error: {e}")
            crime_dist_dfs[dist_id][cluster_name]["model_config"] = None  # If fitting fails

#### Plotting

In [ ]:
def plot_crime_forecast(train_data, test_data, forecast_data, crime_type="Unknown Crime", district_id="Unknown District", plot_train=True):
    """
    Plots the training data, test data, and forecasted data in one plot.
    """
    
    plt.figure(figsize=(20, 6))
    
    if plot_train:
        # Plot training data
        plt.plot(train_data, label="Training Data", color="blue")
    
    # Plot test data
    plt.plot(test_data, label="Test Data (Actual)", color="green")
    
    # Plot forecast data
    plt.plot(forecast_data, label="Predictions", color="red", linestyle="--")
    
    # Formatting the plot
    plt.title(f"Crime Forecast for {crime_type} in District {district_id}")
    plt.xlabel("Date")
    plt.ylabel("Number of Crimes")
    plt.legend(loc="upper left")
    plt.grid(True)
    plt.show()

In [ ]:
district = 1.0
crime_name = 'Auto Theft'
plot_crime_forecast(crime_dist_dfs[district][crime_name]["train"], crime_dist_dfs[district][crime_name]["test"], crime_dist_dfs[district][crime_name]["forecast"], crime_type=crime_name, district_id=district)

In [ ]:
district = 1.0
crime_name = 'Auto Theft'
plot_crime_forecast(crime_dist_dfs[district][crime_name]["train"], crime_dist_dfs[district][crime_name]["test"], crime_dist_dfs[district][crime_name]["forecast"], crime_type=crime_name, district_id=district, plot_train=False)

In [ ]:
def plot_aic_values(crime_dist_dfs):
    # Step 1: Get all districts and crimes
    districts = list(crime_dist_dfs.keys())
    num_districts = len(districts)
    crimes = {crime for district in crime_dist_dfs.values() for crime in district.keys()}
    num_crimes = len(crimes)

    # Step 2: Set up subplots based on number of districts and crimes
    fig, axs = plt.subplots(num_districts, 1, figsize=(10, num_districts * 5))
    fig.suptitle('AIC values for Different Crimes across Districts')

    # Step 3: Populate each subplot
    for idx, district in enumerate(districts):
        aic_values = []
        crime_names = []
        
        for crime in crime_dist_dfs[district].keys():
            aic = crime_dist_dfs[district][crime]["model_config"]["aic"]
            aic_values.append(aic)
            crime_names.append(crime)
        
        ax = axs[idx] if num_districts > 1 else axs  # Adjust if single subplot
        ax.bar(crime_names, aic_values)
        ax.set_title(f'AIC values in {district}')
        ax.set_ylabel('AIC')
        ax.set_xlabel('Crime Type')
        ax.set_xticks(crime_names)
        ax.set_xticklabels(crime_names, rotation=25)  # Rotate x-tick labels
        for i, v in enumerate(aic_values):
            ax.text(i, v, f"{v:.2f}", ha='center', va='bottom', fontsize=8)

    plt.tight_layout(rect=[0, 0.02, 1, 0.98])
    plt.show()

In [ ]:
plot_aic_values(crime_dist_dfs)

In [ ]:
def plot_rmse_heatmap(crime_dist_dfs):
    """
    Function to create a heatmap plot of RMSE values for crimes across districts
    """
    # Initialize an empty dictionary to hold RMSE values
    rmse_data = {
        "District": [],
        "Crime": [],
        "RMSE": []
    }
    
    # Collect RMSE values for each crime in each district
    for dist_id, crimes in crime_dist_dfs.items():
        for crime_name, crime_info in crimes.items():
            if crime_info.get("model_config") and crime_info["model_config"].get("rmse") is not None:
                rmse_data["District"].append(dist_id)
                rmse_data["Crime"].append(crime_name)
                rmse_data["RMSE"].append(crime_info["model_config"]["rmse"])
    
    # Create a DataFrame from the collected RMSE data
    rmse_df = pd.DataFrame(rmse_data)
    rmse_pivot = rmse_df.pivot(index="Crime", columns="District", values="RMSE")
    
    # Plot the heatmap
    plt.figure(figsize=(8, 8))
    sns.heatmap(rmse_pivot, annot=True, fmt=".2f", cmap="coolwarm", cbar_kws={'label': 'RMSE'})
    plt.title("RMSE of Crime Predictions by District and Crime Category")
    plt.xlabel("District")
    plt.ylabel("Crime Category")
    plt.tight_layout()
    plt.show()

In [ ]:
plot_rmse_heatmap(crime_dist_dfs)

In [ ]:
if SAVE_DICT_TO_DISK:
    # Save dictionary to a pickle file
    with open(f"..\\data\\mergedData\\crime_dist_dfs_top_{TOP_X_CRIMES}_{CURRENT_DATE}.pkl", 'wb') as file:
        pickle.dump(crime_dist_dfs, file)

In [ ]:
# Load dictionary from a pickle file
#with open(f"..\\data\\mergedData\\crime_dist_dfs_top_{TOP_X_CRIMES}.pkl", 'rb') as file:
#    crime_dist_dfs = pickle.load(file)

### Automatic pipeline

---